In [ ]:
#The given script reads a video from the disk, estimates the path in accordance to the cone placement, and then 
#sends the appropriate steering angle to an arduino which controls an actuator attached to the steering column

In [1]:
from __future__ import division
from matplotlib import pyplot as plt
import logging
import logging.config
import time
from moviepy.editor import VideoFileClip
from IPython.display import HTML
import cv2
import numpy as np
import tensorflow as tf
import serial #Serial imported for Serial communication
import time #Required to use delay functions
from utils import cv_utils
from utils import operations as ops
from utils import tf_utils
ArduinoSerial = serial.Serial('COM3',9600)
time.sleep(2)
logging.config.fileConfig('logging.ini')

VIDEO_PATH = 'sample_video.mp4'
FROZEN_GRAPH_PATH = 'models/ssd_mobilenet_v1/frozen_inference_graph.pb'

OUTPUT_WINDOW_WIDTH = None  # Use None to use the original size of the image
DETECT_EVERY_N_SECONDS = None  # Use None to perform detection for each frame

# TUNE ME
CROP_WIDTH = CROP_HEIGHT = 600
CROP_STEP_HORIZONTAL = CROP_STEP_VERTICAL = 600 - 20  # no cone bigger than 20px
SCORE_THRESHOLD = 0.5
NON_MAX_SUPPRESSION_THRESHOLD = 0.5


def main():
    # Read TensorFlow graph
    detection_graph = tf_utils.load_model(FROZEN_GRAPH_PATH)

    # Read video from disk and count frames
    #cap = cv2.VideoCapture(0)
    cap = cv2.VideoCapture(VIDEO_PATH)

    fps = cap.get(cv2.CAP_PROP_FPS)

    # CROP_WIDTH = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    # CROP_HEIGHT = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

    with tf.compat.v1.Session(graph=detection_graph) as sess:
        fourcc = cv2.VideoWriter_fourcc(*'XVID')
        out = cv2.VideoWriter('Prototype2.avi', fourcc, 5, (int(cap.get(3)), int(cap.get(4))))
        processed_images = 0
        while cap.isOpened():
            width = cap.get(3)
            height = cap.get(4)
            fps = cap.get(5)
            if DETECT_EVERY_N_SECONDS:
                cap.set(cv2.CAP_PROP_POS_FRAMES,
                        processed_images * fps * DETECT_EVERY_N_SECONDS)
            size = (width, height)
            
            ret, frame = cap.read()
            #frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            if ret:
                tic = time.time()
                i=0
                cv2.line(frame, (int(width/2),int(height/2)), (int(width/2),int(height/2)+40), (255,0,0), 10)
                font = cv2.FONT_HERSHEY_SIMPLEX
                
                # crops are images as ndarrays of shape
                # (number_crops, CROP_HEIGHT, CROP_WIDTH, 3)
                # crop coordinates are the ymin, xmin, ymax, xmax coordinates in
                #  the original image
                crops, crops_coordinates = ops.extract_crops(
                    frame, CROP_HEIGHT, CROP_WIDTH,
                    CROP_STEP_VERTICAL, CROP_STEP_VERTICAL)

                # Uncomment this if you also uncommented the two lines before
                #  creating the TF session.
                # crops = np.array([crops[0]])
                # crops_coordinates = [crops_coordinates[0]]

                detection_dict = tf_utils.run_inference_for_batch(crops, sess)

                # The detection boxes obtained are relative to each crop. Get
                # boxes relative to the original image
                # IMPORTANT! The boxes coordinates are in the following order:
                # (ymin, xmin, ymax, xmax)
                boxes = []
                for box_absolute, boxes_relative in zip(
                        crops_coordinates, detection_dict['detection_boxes']):
                    boxes.extend(ops.get_absolute_boxes(
                        box_absolute,
                        boxes_relative[np.any(boxes_relative, axis=1)]))
                if boxes:
                    boxes = np.vstack(boxes)

                # Remove overlapping boxes
                boxes = ops.non_max_suppression_fast(
                    boxes, NON_MAX_SUPPRESSION_THRESHOLD)

                # Get scores to display them on top of each detection
                boxes_scores = detection_dict['detection_scores']
                boxes_scores = boxes_scores[np.nonzero(boxes_scores)]

                for box, score in zip(boxes, boxes_scores):
                    if score > SCORE_THRESHOLD:
                        i=i+1
                        color = (0, 255, 0) 
                        thickness=10
                        ymin, xmin, ymax, xmax = box
                        if i==1:
                            x1=(xmin+xmax)/2
                            y1=(ymin+ymax)/2
                        elif i==2:
                            x2=(xmin+xmax)/2
                            y2=(ymin+ymax)/2
                            start_point=(int((x1+x2)/2),int(y1))
                            end_point=(int((x1+x2)/2),int(y1)+40)
                            if (x1>(width/2) and x2<(width/2)) or (x1<(width/2) and x2>(width/2)):
                                cv2.line(frame, start_point, end_point, color, thickness)
                                diff=(width/2)-((x1+x2)/2)+160
                                cv2.putText(frame,str(diff),(int(width/2),int(height)-600), font, 3,(255,255,255),2,cv2.LINE_AA)
                                diff_2=str(diff)
                                
                                ArduinoSerial.write(diff_2.encode())
                                print (ArduinoSerial.readline())
                        color_detected_rgb = cv_utils.predominant_rgb_color(
                            frame, ymin, xmin, ymax, xmax)
                        text = '{:.2f}'.format(score)
                        cv_utils.add_rectangle_with_text(
                            frame, ymin, xmin, ymax, xmax,
                            color_detected_rgb, str(i))

                if OUTPUT_WINDOW_WIDTH:
                    frame = cv_utils.resize_width_keeping_aspect_ratio(
                        frame, OUTPUT_WINDOW_WIDTH)

                
                cv2.waitKey(1)
                processed_images += 1

                toc = time.time()
                processing_time_ms = (toc - tic) * 1000
                logging.debug(
                    'Detected {} objects in {} images in {:.2f} ms'.format(
                        len(boxes), len(crops), processing_time_ms))
                cv2.imshow('Live Video', frame)
                out.write(frame)
                #out = cv2.VideoWriter('test_file.avi',cv2.VideoWriter_fourcc('M','J','P','G'), 10, (width,height))
                #if cv2.waitKey(1) & 0xFF == ord('q'):
                    #break
            else:
                # No more frames. Break the loop
                break

    cap.release()
    cv2.destroyAllWindows()


if __name__ == '__main__':
    main()

b'Interrupt attached.\r\n'


2020-02-12 19:42:01,982 - root - DEBUG - Detected 8 objects in 6 images in 4124.65 ms


b'Calculated output rot:\r\n'


2020-02-12 19:42:02,918 - root - DEBUG - Detected 8 objects in 6 images in 854.53 ms


b'13\r\n'


...

2020-02-12 19:43:46,051 - root - DEBUG - Detected 7 objects in 6 images in 780.44 ms


b'0\r\n'


2020-02-12 19:43:46,834 - root - DEBUG - Detected 7 objects in 6 images in 755.98 ms


b'0\r\n'


2020-02-12 19:43:47,606 - root - DEBUG - Detected 6 objects in 6 images in 746.43 ms
